In [1]:
from logger import FileProcessLogger, file_line_count

In [2]:
def get_bigram():
    filename = './dane/poleval_2grams.txt'
    bigram = {}
    with open(filename, 'r', encoding='utf-8') as file:
        logger = FileProcessLogger(filename)
        for line in file:
            logger.update()
            n, word1, word2 = line.lower().split()
            n = int(n)
            if n < 10: 
                continue
            if word1 in bigram:
                bigram[word1][word2] = n
            else:
                bigram[word1] = { word2: n }
    return bigram

In [3]:
bigrams = get_bigram()

Processed 10% of the file contents...
Processed 20% of the file contents...
Processed 30% of the file contents...
Processed 40% of the file contents...
Processed 50% of the file contents...
Processed 60% of the file contents...
Processed 70% of the file contents...
Processed 80% of the file contents...
Processed 90% of the file contents...
Processed 100% of the file contents...


In [4]:
tags = {}
with open('./dane/supertags.txt', 'r', encoding='utf-8') as file:
    for line in file:
        word, tag = line.rstrip('\n').split(' ')
        tags[word] = tag

In [5]:
bigram_tags = {}
with open('./dane/poleval_2grams.txt', 'r', encoding='UTF-8') as file:
    for line in file:
        n, word1, word2 = line.lower()[:-1].split(' ')
        if word1 in tags and word2 in tags:
            n, tag1, tag2 = int(n), tags[word1], tags[word2]
            if tag1 not in bigram_tags:
                bigram_tags[tag1] = { tag2: n }
            else:
                if tag2 not in bigram_tags[tag1]:
                    bigram_tags[tag1][tag2] = 0
                bigram_tags[tag1][tag2] += n

In [6]:
from math import log as ln

In [7]:
def popular_words(bigrams, skip, take):
    words = [(word, sum(bigrams[word].values())) for word in bigrams]
    return sorted(words, key=lambda x: x[1], reverse=True)[skip:skip+take]

In [8]:
print(popular_words(bigrams, 1000, 10))

[('uzyskał', 35775), ('posiedzeniu', 35771), ('każdej', 35706), ('zmarł', 35687), ('stało', 35666), ('francji', 35602), ('procesu', 35554), ('oni', 35518), ('poziomu', 35510), ('zjednoczonych', 35396)]


In [9]:
def PPMI(x, y):
    Px = 0
    Py = 0
    Pxy = 0.001

    if x in bigrams:
        Px = sum(bigrams[x].values())
        if y in bigrams[x]:
            Pxy = bigrams[x][y]
    for word in bigrams:
        if x in bigrams[word]:
            Px += bigrams[word][x]
     
    if y in bigrams:
        Py = sum(bigrams[y].values())
    for word in bigrams:
        if y in bigrams[word]:
            Py += bigrams[word][y]

    N = 383345990
    PMI = ln(N * Pxy / Px / Py)
    return max(0, PMI) 

In [10]:
def PSM(x, y):
    Px = 0
    Py = 0
    Pxy = 0.001

    if x in bigrams:
        Px = sum(bigrams[x].values())
        if y in bigrams[x]:
            Pxy = bigrams[x][y]
    for word in bigrams:
        if x in bigrams[word]:
            Px += bigrams[word][x]
     
    if y in bigrams:
        Py = sum(bigrams[y].values())
    for word in bigrams:
        if y in bigrams[word]:
            Py += bigrams[word][y]
    
    N = 383345990
    estimatedP = Px * Py / N

    return Pxy * (ln(Pxy) - ln(estimatedP) - 1)

In [13]:
def get_tag(x):
    if x in tags:
        return tags[x]
    suffix = x[-3:]
    if suffix in tags:
        return tags[suffix]
    return None


def tagged_PPMI(x, y):
    tag_x = get_tag(x)
    if tag_x is None: return 0
    tag_y = get_tag(y)
    if tag_y is None: return 0
    P_tag_x = 0
    P_tag_y = 0
    P_tags = 0.001

    if tag_x in bigram_tags:
        P_tag_x = sum(bigram_tags[tag_x].values())
        if tag_y in bigram_tags[tag_x]:
            P_tags = bigram_tags[tag_x][tag_y]
    for word in bigram_tags:
        if tag_x in bigram_tags[word]:
            P_tag_x += bigram_tags[word][tag_x]
     
    if y in bigram_tags:
        P_tag_y = sum(bigram_tags[tag_y].values())
    for word in bigram_tags:
        if tag_y in bigram_tags[word]:
            P_tag_y += bigram_tags[word][tag_y]

    N = 383345990
    PMI = ln(N * P_tags / P_tag_x / P_tag_y)
    return max(0, PMI) * PPMI(x, y)

In [14]:
methods = [('PPMI',PPMI), ('PSM', PSM), ('tagged_PPMI', tagged_PPMI)]
for word, _ in popular_words(bigrams, 1000, 10):
    print(f'testing for word: "{word}"')
    for name, method in methods:
        print()
        print('Method:', name)
        scores = [(method(word, successor), successor) for successor in bigrams[word].keys()]
        scores.sort(key=lambda score: score[0], reverse=True)
        for (score, successor) in scores[:10]:
            print(word , successor, score)
    print(20 * '-')

testing for word: "uzyskał"

Method: PPMI
uzyskał reelekcji 7.210127276806494
uzyskał bakalaureat 7.0995949727462735
uzyskał licencjat 7.016188217012809
uzyskał habilitację 6.815102527492896
uzyskał magisterium 6.7634162710211285
uzyskał mandat 6.653081042942986
uzyskał doktorat 6.552324052563292
uzyskał doktoraty 6.529281499549021
uzyskał dyplom 6.348639483936616
uzyskał profesurę 6.273917371176575

Method: PSM
uzyskał mandat 20695.929698214273
uzyskał stopień 17850.419369789994
uzyskał tytuł 14695.222506511493
uzyskał doktorat 6124.213429977311
uzyskał dyplom 5969.081664073263
uzyskał status 3276.1616672223213
uzyskał reelekcję 2872.489998525407
uzyskał licencjat 2027.4554291333168
uzyskał habilitację 1872.4630138527125
uzyskał magisterium 1711.7346324932755

Method: tagged_PPMI
uzyskał habilitację 11.210583041119309
uzyskał profesurę 10.320354154462024
uzyskał reelekcję 9.694597535059321
uzyskał akceptację 9.20411803603595
uzyskał nominację 9.033028400651949
uzyskał licencjat 8.7290